In [1]:
# add repo path to the system path
from pathlib import Path
import os, sys
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None
sys.path.insert(0,str(repo_path / 'detr')) if str(repo_path / 'detr') not in sys.path else None

os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [2]:
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode
from detectron2.structures import pairwise_iou, boxes
from detectron2.structures import BoxMode
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, launch
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_train_loader
from detectron2.solver.build import maybe_add_gradient_clipping

from typing import Any, Dict, List, Set
import itertools

import torch
import cv2 as cv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

Trainer function

In [3]:
class Trainer(DefaultTrainer):
    """
    Extension of the Trainer class adapted to DETR.
    """

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        """
        Create evaluator(s) for a given dataset.
        This uses the special metadata "evaluator_type" associated with each builtin dataset.
        For your own dataset, you can simply create an evaluator manually in your
        script and do not have to worry about the hacky if-else logic here.
        """
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, True, output_folder)

    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=None)

    @classmethod
    def build_optimizer(cls, cfg, model):
        params: List[Dict[str, Any]] = []
        memo: Set[torch.nn.parameter.Parameter] = set()
        for key, value in model.named_parameters(recurse=True):
            if not value.requires_grad:
                continue
            # Avoid duplicating parameters
            if value in memo:
                continue
            memo.add(value)
            lr = cfg.SOLVER.BASE_LR
            weight_decay = cfg.SOLVER.WEIGHT_DECAY
            if "backbone" in key:
                lr = lr * cfg.SOLVER.BACKBONE_MULTIPLIER
            params += [{"params": [value], "lr": lr, "weight_decay": weight_decay}]

        def maybe_add_full_model_gradient_clipping(optim):  # optim: the optimizer class
            # detectron2 doesn't have full model gradient clipping now
            clip_norm_val = cfg.SOLVER.CLIP_GRADIENTS.CLIP_VALUE
            enable = (
                cfg.SOLVER.CLIP_GRADIENTS.ENABLED
                and cfg.SOLVER.CLIP_GRADIENTS.CLIP_TYPE == "full_model"
                and clip_norm_val > 0.0
            )

            class FullModelGradientClippingOptimizer(optim):
                def step(self, closure=None):
                    all_params = itertools.chain(*[x["params"] for x in self.param_groups])
                    torch.nn.utils.clip_grad_norm_(all_params, clip_norm_val)
                    super().step(closure=closure)

            return FullModelGradientClippingOptimizer if enable else optim

        optimizer_type = cfg.SOLVER.OPTIMIZER
        if optimizer_type == "SGD":
            optimizer = maybe_add_full_model_gradient_clipping(torch.optim.SGD)(
                params, cfg.SOLVER.BASE_LR, momentum=cfg.SOLVER.MOMENTUM
            )
        elif optimizer_type == "ADAMW":
            optimizer = maybe_add_full_model_gradient_clipping(torch.optim.AdamW)(
                params, cfg.SOLVER.BASE_LR
            )
        else:
            raise NotImplementedError(f"no optimizer type {optimizer_type}")
        if not cfg.SOLVER.CLIP_GRADIENTS.CLIP_TYPE == "full_model":
            optimizer = maybe_add_gradient_clipping(cfg, optimizer)
        return optimizer

In [8]:
DatasetCatalog.clear()

# parameters
iter_CEM = 30000
p_CEM = "30k"

# tests:. 101 vs R2
m_layer = 'R_101'
g_rgb = "gray" # if we train with rgb or grayscale
csv_dbt_file= "../train_bboxes_gray_fold2.csv"
dbt_dicts = "/home/robert/data/DBT2/train_gray"

In [ ]:
def get_omidb_dicts(img_dir):
    
    csv_file = os.path.join(img_dir, "omidb-selection.csv")
    
    df = pd.read_csv(csv_file)
    
    dataset_dicts = []
    for idx, row in df.iterrows():
        record = {}
        skip_file = False
        if (row["type"] == 'M'):
            filename = os.path.join(img_dir+"/HOLOGIC/ffdm/st"+"{0:03}".format(row["subtype"]), row["filename"])
        if (row["type"] == 'MU'):
            filename = os.path.join(img_dir+"/HOLOGIC/ffdm/stu", row["filename"])
        if (row["type"] == 'B'):
            filename = os.path.join(img_dir+"/HOLOGIC/ffdm/benign", row["filename"])
        if (row["type"] == 'N'):
            filename = os.path.join(img_dir+"/HOLOGIC/ffdm/normal", row["filename"])
#             skip_file = True
        
        if (not skip_file):

            # its slow, reading all the images to know dimensions!         
    #         height, width = cv2.imread(filename).shape[:2]

            record["file_name"] = filename
            record["image_id"] = idx

            # Bounding box breast area         
            bbox = row["bbox"][12:-1]
            coords = bbox.split(',')
            r= np.array([0,0,0,0])
            idx = 0
            for c in coords:
                aux = c.split('=')
                r[idx]=(int(aux[1]))
                idx +=1

            # we can get width and heigth from bbox
            record["height"] = r[3]-r[1]
            record["width"] = r[2]-r[0]

            if (row["type"] == 'N'): 
                record["annotations"] = []
            else:
                # Bounding box roi  
                bbox_roi = row["bbox_roi"][12:-1]
                coords = bbox_roi.split(',')
                s= np.array([0,0,0,0])
                idx = 0
                for c in coords:
                    aux = c.split('=')
                    s[idx]=(int(aux[1]))
                    idx +=1
                bbox_roi = omidb.mark.BoundingBox(s[0]-r[0],s[1]-r[1],s[2]-r[0],s[3]-r[1])

                px = [bbox_roi.x1, bbox_roi.x2, bbox_roi.x2, bbox_roi.x1]
                py = [bbox_roi.y1, bbox_roi.y1, bbox_roi.y2, bbox_roi.y2]
                poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
                poly = [p for x in poly for p in x]
                objs = []
                obj =  {
                        "bbox": [bbox_roi.x1 , bbox_roi.y1, bbox_roi.x2, bbox_roi.y2],
                        "bbox_mode": BoxMode.XYXY_ABS,
                        "segmentation": [poly],
                        "category_id": 0,
                    }
                objs.append(obj)
                record["annotations"] = objs
            dataset_dicts.append(record)
    return dataset_dicts

DatasetCatalog.register("omidb_train", lambda: get_omidb_dicts("/mnt/mia_images/breast/iceberg_selection2"))
MetadataCatalog.get("omidb_train").set(thing_classes=["lesion"])
omidb_metadata = MetadataCatalog.get("omidb_train")